
## Autograded Notebook (Canvas & CodeGrade)

This notebook will be automatically graded. It is designed to test your answers and award points for the correct answers. Following the instructions for each Task carefully.
Instructions

- **Download** this notebook as you would any other ipynb file 
- **Upload** to Google Colab or work locally (if you have that set-up)
- **Delete** `raise NotImplementedError()`

- **Write** your code in the `# YOUR CODE HERE` space


- **Execute** the Test cells that contain assert statements - these help you check your work (others contain hidden tests that will be checked when you submit through Canvas)

- **Save** your notebook when you are finished
- **Download** as a ipynb file (if working in Colab)
- **Upload** your complete notebook to Canvas (there will be additional instructions in Slack and/or Canvas)



# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
Successfully complete all these objectives to earn full credit. 

**Successful completion is defined as passing all the unit tests in each objective.**  

Each unit test that you pass is 1 point. 

There are 5 total possible points in this sprint challenge. 


There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

____

# Before you submit your notebook you must first

1) Restart your notebook's Kernal

2) Run all cells sequentially, from top to bottom, so that cell numbers are sequential numbers (i.e. 1,2,3,4,5...)
- Easiest way to do this is to click on the **Cell** tab at the top of your notebook and select **Run All** from the drop down menu. 

3) Comment out the cell that generates a pyLDAvis visual in objective 4 (see instructions in that section). 
____



### Import Data

In [1]:
import pandas as pd

# Load reviews from URL
data_url = 'https://raw.githubusercontent.com/LambdaSchool/data-science-practice-datasets/main/unit_4/unit1_nlp/review_sample.json'

# Import data into a DataFrame named df
data = pd.read_json(data_url, lines=True)
df = pd.DataFrame(data=data)

In [2]:
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [3]:
# Visible Testing
assert isinstance(df, pd.DataFrame), 'df is not a DataFrame. Did you import the data into df?'
assert df.shape[0] == 10000, 'DataFrame df has the wrong number of rows.'

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [4]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 12.6 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [5]:
# Optional: Consider using spaCy in your function. The spaCy library can be imported by running this cell.
# A pre-trained model (en_core_web_sm) has been made available to you in the CodeGrade container.
# If you DON'T need use the en_core_web_sm model, you can comment it out below.

import en_core_web_sm
nlp = en_core_web_sm.load()

In [6]:
def tokenize(doc):
    """ Function that tokenizes text
    Parameters
    ------
    doc: str
    ------
    Returns
    ------
    list of tokens
    """
    doc_tokens = []
    for token in nlp(doc):
        if (token.is_digit == False) and (token.is_digit == False) \
            and (token.is_punct == False) and (token.is_space == False) \
            and (token.is_stop == False) and (token.pos_ != 'PRON'):
                doc_tokens.append(token.text.lower())
    return doc_tokens

In [7]:
'''Testing'''
assert isinstance(tokenize(df.sample(n=1)["text"].iloc[0]), list), "Make sure your tokenizer function accepts a single document and returns a list of tokens!"

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews (i.e. create a doc-term matrix).
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, use `NearestNeighbors` model for this. 

In [8]:
df['text'].sample(6)

4274    We went to Prosecco on Friday night for dinner...
2602    Great food. I'm trying to promote for you but ...
2526    This has become a must do for my girlfriend an...
481     I have been taking my dogs here for years and ...
4150    This place is great if you like ordering food ...
7609    I dropped in for lunch, and sadly I wasn't sup...
Name: text, dtype: object

In [9]:
# Create a vector representation of the reviews 
# Name that doc-term matrix "dtm"

from sklearn.feature_extraction.text import TfidfVectorizer
# tokenize text

df['text_cleaned'] = df['text'].apply(lambda x: ' '.join(tokenize(x)))

tfidf = TfidfVectorizer(stop_words='english',
                       ngram_range=(1, 2),
                       min_df=2,
                       max_df=.35)

dtm = tfidf.fit_transform(df['text_cleaned'])
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

In [10]:
# Create and fit a NearestNeighbors model named "nn"
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [11]:
'''Testing.'''
assert nn.__module__ == 'sklearn.neighbors._unsupervised', ' nn is not a NearestNeighbors instance.'
assert nn.n_neighbors == 10, 'nn has the wrong value for n_neighbors'

In [13]:
# Create a fake review and find the 10 most similar reviews

fake_review = 'Lovely cheap tiny place. Would definetly recommend visiting. Hop in for lunch.'
fake_review = tfidf.transform([' '.join(tokenize(fake_review))])

most_similar_revs = nn.kneighbors(fake_review.todense())[1][0]

print('Ten most similar reviews are:')
for rev_num in most_similar_revs:
    print('------------------')
    print(df['text'][rev_num])
    print('------------------')

Ten most similar reviews are:
------------------
旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした。
質問にも丁寧に答えてくれましたし、日本人の方も日本語が話せる方も居て、とても綺麗で居心地のいいお店でした。 
ネイルはちはるさんと言う綺麗な方が丁寧にしてくれとても気に入りました。
予定になかったまつ毛エクステもお願いし、日本ではまだあまりないブラウンカラーのエクステをしてもらい、とても気に入りました。
また是非マッサージなどで伺いたいと思います。
------------------
------------------
天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用心製作，包含擺盤精緻、佐料衛生，夏日想開胃，這是一個不錯的選擇，服務人員也很敬業，以後會常常來
------------------
------------------
Honestly one of the best BBQ places I've ever been. Not only was the food absolutely amazing, but the service was superb! I highly recommend anyone visiting to eat here.
------------------
------------------
Definetly recommend this nail salon ! Gave me exactly what I asked for. I'd give 5 stars but I was rushed once I walked in and I wasn't able to tip because I only had card. There was no wait lots of colors to choose from and the staff was beyond friendly. I would most certainly come back !
------------------
------------------
Gino (aka jean machine) is up

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a pipeline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier.
    - Use that pipeline to train a model to predict the `stars` feature (i.e. the labels). 
    - Use that Pipeline to predict a star rating for your fake review from Part 2. 



2. Create a parameter dict including `one parameter for the vectorizer` and `one parameter for the model`. 
    - Include 2 possible values for each parameter
    - **Use `n_jobs` = 1** 
    - Due to limited computational resources on CodeGrader `DO NOT INCLUDE ADDITIONAL PARAMETERS OR VALUES PLEASE.`
    
    
3. Train the entire pipeline with a GridSearch
    - Name your GridSearch object as `gs`

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

# Name the gridsearch instance "gs"

# vectorizer
vect = TfidfVectorizer(stop_words='english',
                       ngram_range=(1, 2),
                       min_df=2)

# classifier
clf = LinearSVC(penalty='l2', 
                loss='hinge')

# pipeline
pipe = Pipeline([('vect', vect), ('clf', clf)])

# parameters
params = {
    'vect__max_df':(.35, .75),
    'clf__C':(1., 2.)
}

# gridsearch
gs = GridSearchCV(pipe, params, cv=5, n_jobs=1)
gs.fit(df['text_cleaned'], df['stars'])

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=2,
                                                        ngram_range=(1, 2),
                                                        no

In [15]:
# Visible Testing
prediction = gs.predict(["I wish dogs knew how to speak English."])[0]
assert prediction in df.stars.values, 'You gs object should be able to accept raw text within a list. Did you include a vectorizer in your pipeline?'

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Set num_topics to `5`
    - Name your LDA model `lda`
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

When you instantiate your LDA model, it should look like this: 

```python
lda = LdaModel(corpus=corpus,
               id2word=id2word,
               random_state=723812,
               num_topics = num_topics,
               passes=1
              )

```

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

## Note about  pyLDAvis

**pyLDAvis** is the Topic modeling package that we used in class to visualize the topics that LDA generates for us.

You are welcomed to use pyLDAvis if you'd like for your visualization. However, **you MUST comment out the code that imports the package and the cell that generates the visualization before you submit your notebook to CodeGrade.** 

Although you should leave the print out of the visualization for graders to see (i.e. comment out the cell after you run it to create the viz). 

In [16]:
from gensim import corpora
# Due to limited computationalresources on CodeGrader, use the non-multicore version of LDA 
from gensim.models.ldamodel import LdaModel
import gensim
import re

### 1. Estimate a LDA topic model of the review tex

In [17]:
# Remember to read the LDA docs for more information on the various class attirbutes and methods available to you
# in the LDA model: https://radimrehurek.com/gensim/models/ldamodel.html

# don't change this value 
num_topics = 5

# use tokenize function you created earlier to create tokens
df['text_tokenized'] = df['text'].apply(lambda x: tokenize(x))

# create a id2word object (hint: use corpora.Dictionary)
id2word = corpora.Dictionary(df['text_tokenized'])

# create a corpus object (hint: id2word.doc2bow)
corpus = [id2word.doc2bow(doc) for doc in df['text_tokenized']]

# instantiate an lda model
lda = LdaModel(corpus=corpus,
               id2word=id2word,
               random_state=723812,
               num_topics = num_topics,
               passes=1
              )


#### Testing

In [18]:
# Visible Testing
assert lda.get_topics().shape[0] == 5, 'Did your model complete its training? Did you set num_topics to 5?'

#### 2. Create 1-2 visualizations of the results

In [19]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

# Use pyLDAvis (or a ploting tool of your choice) to visualize your results 

pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.130373  0.009338       1        1  41.983994
3      0.073905  0.072278       2        1  17.462666
0      0.000183 -0.019588       3        1  16.987884
2     -0.008876  0.001065       4        1  12.764766
4      0.065161 -0.063093       5        1  10.800690, topic_info=        Term         Freq        Total Category  logprob  loglift
35      time  3402.000000  3402.000000  Default  30.0000  30.0000
145     food  5081.000000  5081.000000  Default  29.0000  29.0000
212    great  4378.000000  4378.000000  Default  28.0000  28.0000
398    pizza   813.000000   813.000000  Default  27.0000  27.0000
0          $  2169.000000  2169.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
212    great   180.254826  4378.972715   Topic5  -5.6690  -0.9646
1086     new   134.795917  1105.023844   Topic5  -5.9596   0.1217
541     work   126.870068   919.972737   Topic5  -6.0202   0.2444
129   people   131.404327  1439.917148   Topic5  -5.9851  -0.1685
120     best   113.496295  1822.258678   Topic5  -6.1316  -0.5505

[466 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
0          1  0.382084      $
0          2  0.100937      $
0          3  0.175141      $
0          4  0.194038      $
0          5  0.147487      $
...      ...       ...    ...
2611       5  0.020044  yummy
12099      3  0.861249    zen
4127       1  0.020572      à
4127       3  0.020572      à
4127       5  0.946325      à

[1183 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3, 5])

#### 3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model
LDA divided documents into 5 major topics based on the reviews customers have given to the place:
- first largest group is the one with 5 star reviews
- second - 4 star reviews
- third and forth are mostly identical - these are reviews with 3 or 2 stars, they are sharing a lot of semantic meaning
- last smallest group is the one with one star reviews.
By setting alpha to a larger number we can see words, that are common for some topic, but that still appear a lot in
the corpus, when alpha is low very specific words pop out, that may reflect peculiarities of a certain topic

On the plot we can see that topics are overall evenly distributed.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)